In [ ]:
!pip install langchain
!pip install langchain_community
!pip install langchain_chroma
!pip install langchainhub



INDEXING:LOAD

LOADING THE CONTENTS FROM POST TO A TEXT FORMAT

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer('div')
loader=WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Studio_Ghibli",),
    bs_kwargs={"parse_only": bs4_strainer},
)

docs= loader.load()

len(docs[0].page_content)

print(docs[0].page_content[:500])








Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search





























Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk



























Contents
move to sidebar
hi


INDEXING:SPLIT

SPLITTING DOCUMENT INTO CHUNKS TO FIT INTO CONTEXT WINDOW OF MODELS

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
print(type(all_splits))
print(all_splits)

<class 'list'>
[Document(page_content='Main menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1Name\n\n\n\n\n\n\n\n2History\n\n\n\nToggle History subsection\n\n\n\n\n\n2.1Tokuma Shoten era\n\n\n\n\n\n\n\n2.2Independent era\n\n\n\n\n\n\n\n2.3Nippon TV era\n\n\n\n\n\n\n\n\n\n3Distribution rights\n\n\n\nToggle Distribution rights subsection\n\n\n\n\n\n3.1Theatrical and home media 

INDEXING:store

STORING DATA INTO CHROMA VECTOR STORE AND OPENAI EMBEDDINGS MODEL

In [3]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
##api_key = os.getenv('HF_API_KEY')

embeddings = HuggingFaceInferenceAPIEmbeddings(    
            api_key=os.getenv('HF_API_KEY'), 
            model_name="thenlper/gte-large"
 )
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)


RETRIEVAL AND GENERATION: RETRIEVAL

In [5]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

len(retrieved_docs)

print(retrieved_docs[0].page_content)

with LCCN identifiersArticles with NDL identifiersArticles with NKC identifiersArticles with NLA identifiersArticles with CINII identifiersArticles with ULAN identifiersArticles with SUDOC identifiers


RETRIEVAL AND GENERATION: GENERATION

In [13]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI
import os 
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('GCP_API_KEY')

llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)

prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("grave of the fireflies")

'Grave of the Fireflies was released in Japan on VHS by Buena Vista Home Entertainment under the Ghibli ga Ippai Collection on August 7, 1998.'